In [1]:
import random

# Algoritmos de Euclides

Algoritmo de Euclides para determinar el máximo común divisor (gcd) de dos enteros a yb

In [2]:
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

print('gcd(2, 3) = ', gcd(2, 3))
print('gcd(20, 30) = ', gcd(20, 30))
print('gcd(50720, 48184) = ', gcd(50720, 48184))


gcd(2, 3) =  1
gcd(20, 30) =  10
gcd(50720, 48184) =  2536


Algoritmo generalizado de Euclides para encontrar el inverso multiplicativos de un número en un anillo cíclico $\mathbb{Z}_{\phi}$

In [4]:
def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi
    
    while e > 0:
        temp1 = temp_phi // e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2
        
        x = x2 - temp1 * x1
        y = d - temp1 * y1
        
        x2 = x1
        x1 = x
        d = y1
        y1 = y

    if temp_phi == 1:
        return d + phi
    # no inverse: return None
    return None

print('3^{-1} mod 10 = ', multiplicative_inverse(3, 10))
print('2^{-1} mod 10 = ', multiplicative_inverse(2, 10))
print('25^{-1} mod 119 = ', multiplicative_inverse(25, 119))

3^{-1} mod 10 =  7
2^{-1} mod 10 =  None
25^{-1} mod 119 =  100


In [5]:
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True

for i in [2, 5, 19, 25, 222, 314, 317]:
    print(f'{i}: ', is_prime(i))

2:  True
5:  True
19:  True
25:  False
222:  False
314:  False
317:  True


# RSA

RSA son unas pocas funciones sencillas:

- Generación de claves
- Cifrado y descifrado son iguals (y simplemente es una potencia)

In [6]:
def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    #n = pq
    n = p * q

    #Phi is the totient of n
    phi = (p - 1) * (q - 1)

    # Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    # Use Euclid's Algorithm to verify that e and phi(n) are coprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)
    
    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

def encrypt(pk, number):
    # Unpack the key into it's components
    key, n = pk
    return (number ** key) % n

decrypt = encrypt

In [8]:
pk, sk = generate_keypair(17, 23)
print(f'Publickey (e, n): {pk} Private-key (d, n): {sk}')

Publickey (e, n): (179, 391) Private-key (d, n): (411, 391)


Fíjate: si generamos otro par de claves, aunque usemos los mismos primos, obtendremos unas claves diferentes. Eso es porque el parámetro $e$ se escoge al azar

In [15]:
pk, sk = generate_keypair(17, 23)
print(f'Publickey: {pk} Private-key: {sk}')

Publickey: (335, 391) Private-key: (207, 391)


Vamos a intentar cifrar un texto sencillo:

In [17]:
print(encrypt(pk, 'hola'))

TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

No podemos: RSA solo puede cifrar enteros. Una posibilidad es codificar el mensaje como un conjunto de enteros

In [18]:
print([encrypt(pk, ord(c)) for c in 'hola'])

[179, 172, 190, 112]


¿Qué pasa si intentamos cifrar varias veces lo mismo?

In [19]:
print([encrypt(pk, ord(c)) for c in 'aaaa'])

[112, 112, 112, 112]


# (semi) Homorfismo

RSA es semihomomórfico con la multiplicación: se pueden hacer cálculos con los números cifrados, aunque no sepas lo que son ni qué resultado tienes. Al descifrar, el resultado es correcto.

Por ejemplo, vamos a multiplicar los mensajes cifrados c1 y c2, que son los cifrados de 5 y 2 respectivamente

In [20]:
m1 = 5
c1 = encrypt(pk, m1)
print(f'encrypt(pk, {m1}) = {c1}')
print(f'decrypt(sk, {c1}) = {decrypt(sk, c1)}')


encrypt(pk, 5) = 296
decrypt(sk, 296) = 5


In [22]:
m2 = 2
c2 = encrypt(pk, m2)
print(f'encrypt(pk, {m2}) = {c2}')
print(f'decrypt(sk, {c2}) = {decrypt(sk, c2)}')

encrypt(pk, 2) = 9
decrypt(sk, 9) = 2


In [25]:
cm = c1 * c2
print(f"c1 = {c1}; c2 = {c2}; cm = {cm}")

c1 = 296; c2 = 9; cm = 2664


Un atacante no sabe cuánto vale c1 ni c2, ni sabe qué valor tiene cm, pero sabe que, sea lo que sea, ha multiplicado c1 y c2 y cuando se descifre el resultado va a ser correcto:

In [27]:
print(f'decrypt(sk, c1 * c2) = m1 * m2 = {m1} * {m2} = {decrypt(sk, cm)}')

decrypt(sk, c1 * c2) = m1 * m2 = 5 * 2 = 10


# PyCryptoDome

La función de arriba solo sirve para ver cómo funciona RSA a alto nivel. Veamos ahora cómo de grandes son los números involucrados en

In [32]:
from Crypto.PublicKey import RSA
key = RSA.generate(4096)
key

RsaKey(n=7146028800751755209495262501603091771029007655411251032058866252716730862993586418564870464861299632072017703618258503350267690104852742323530499194252706345214514787836327917708734913271250233351768251049967890868701681646631027020999262474040057363906993716023053666579627641486290589324737823656100871780246280624175628918069223348100564248752688518074081313899883608776678902517344501765213690422842678903184190142247961689102692871618965449551467267142050229095416811965369717752034786053378598308307901849030604441027533188431735388583170978598022026517611000671222711347802028064699147202832696866127946505317668818315749387372240459105139577832817818160252189131121619870467581916680901773926344787295759033194146134552493423212293274443414070757181911054008552135679780747213981536472188521098228779911348396501280028474686270978532637881027635509579316813715163241203765151654506860147778646749089767986790971321290908654809983552065515017156501027390570706984189510499242906970246

# Ejercicios

Hemos visto cómo crear claves con PyCryptoDrome, pero... 

- ¿Puedes cifrar algo con PyCryptoDrome usando RSA?
- ¿Y un mensaje más largo?
- ¿Cómo se firma?